이 글은 모두의 연구소 AIFFEL Going Deeper node를 참고하여 만들었음을 밝힙니다.


OCR
===


### OutLine

**OCR** = `Text detection` + `Text recognition`
1) `Text detection` ::  문자의 영역을 검출 <br/>
    - object detection, segmentation, 문자가 가지는 독특한 특성을 감안한 형태

2) `Text recognition` :: 검출된 영역의 문자를 인식 <br/>
    - OCR분야에 사용되는 독특한 모델구조를 사용
    

## Tesseract OCR
---

딥러닝이 OCR에 활용되기 전 사용한 방식으로 Tesseract OCR이 있다.


아래의 그림은 OCR엔진에서 Tesseract OCR을 사용하는 그림이다.
![스크린샷, 2021-04-22 10-01-48](https://user-images.githubusercontent.com/70703320/115640687-83464b80-a352-11eb-8e5c-37862a9b92f4.png)


Tesseract OCR의 구조는 크게 5가지 단계로 나뉘어 있다.
![스크린샷, 2021-04-22 10-12-43](https://user-images.githubusercontent.com/70703320/115641092-5b0b1c80-a353-11eb-9031-8cb22756ab1c.png)

1~3번째 단계 :: 입력이미지 추출과 전처리
- `Adaptive Thresholding` ::  입력영상의 이진화를 통해 우리가 흑백으로 보듯이 변환
- `Connected Component Analysis` :: 문자영역을 검출
- `Find Lines and Words` :: 라인 또는 워드 단위를 추출

4번째 단계 :: OCR 처리
- `Recognize Word 1` :: Word 단위 이미지를 Text로 변환하기 위해 문자를 하나씩 인식
5번째 단계 :: OCR의 출력 텍스트의 후처리
- `Recognize Word 2` :: 다시 결합



## Text detection

문자는 몇개가 모여서 단어 혹은 문장을 이루고 있기 때문에 이미지 내에서 문자를 검출해낼 때엔 검출하기 위한 **최소 단위**를 정해야 한다.

![image](https://user-images.githubusercontent.com/70703320/115658448-c5cc5000-a373-11eb-9d35-5f6c53159d5d.png)

방식

- 가로방향(Horizontal)으로만 텍스트 박스를 구하는 방식
- 기울어지거나 세로방향 등 다양한 방향(Multi-oriented)의 텍스트 박스를 구하는 방식


(e)의 경우 :: 전체 파이프라인의 길이가 짧고 간결해서 빠르면서도 정확한 Text detection 성능을 보인다.



### 탐지방식

**단어 단위의 탐지** 
- Object detection의 Regression기반의 Detection방법
- Anchor를 정의하고 단어의 유무, 그리고 Bounding box의 크기를 추정해서 단어를 찾아냄

**글자 단위의 탐지**
- Bounding box regression을 하는 대신 글자인 영역을 Segmentation하는 방법


### 1) regression :: [TextBoxes 논문](https://arxiv.org/pdf/1512.02325.pdf)


- 딥러닝 기반의 Detection을 이용하여 단어단위로 인식
- 네트워크의 기본 구조 ::  `SSD: single shot multibox detector`

![스크린샷, 2021-04-22 14-23-20](https://user-images.githubusercontent.com/70703320/115659987-5310a400-a376-11eb-9c92-1310308fbfb5.png)

- 일반적인 단어들은 가로가 길므로 Aspect ratio가 크다.
- 이 때문에 Regression을 위한 Convolution layer에서 3x3 크기의 kernel이 아닌 1x5 크기의 kernel을 갖는다. 
- Anchor box또한 1,2,3,5,7로 큰 aspect ratio로 만들고 이를 vertical offset을 적용하여 세로 방향으로 촘촘한 단어의 배열에 대응

![image](https://user-images.githubusercontent.com/70703320/115660194-a125a780-a376-11eb-91ef-4a0d152d968b.png)

#### Grid cell의 중앙을 기점을 생성할 경우
- 파란색과 검은색 박스

#### 수직방향으로 옮겨서 촘촘하게 만들어준 것
- 빨간색과 녹색 박스
- 이를 통해 세로로 촘촘하게 anchor box를 배치할 수 있게 된다.




### 2) segmentation :: [PixelLink](https://arxiv.org/pdf/1801.01315.pdf)

#### 나오게 된 배경

Semantic segmentation을 이미지 내의 영역을 class로 분리해내는 방법뿐 아니라 **문자의 영역을 찾기 위해서 적용하면, 배경과 글자인 영역으로 분리**할 수 있다.


문자들은 매우 촘촘히 배열이 되어있으니 글자영역으로 찾아낸 뒤에 이를 분리해내는 작업이나 연결하는 작업을 더해서 원하는 최소단위로 만들어줘야 한다.
 
#### PixelLink 
- `Text 영역을 찾아내는 segmentation` + `어느 방향으로 연결되는지` 같이 학습을 하여 Text 영역간의 분리 및 연결을 할 수 있는 정보를 추가적으로 활용

#### 전체적인 구조 :: U-Net과 비슷하지만 output으로 9가지 정보를 얻는다.
![image](https://user-images.githubusercontent.com/70703320/115661636-b3a0e080-a378-11eb-8bc1-d486344d5e62.png) 

#### output 
- `Text/non-text Prediction을 위한 class segmentation map` :: 해당 영역이 Text인지 Non-text인지 예측값을 의미하는 2개의 커널을 가진다.
- `나머지 8가지` :: 글자의 Pixel을 중심으로 인접한 8개의 Pixel에 대한 연결여부를 의미하는 16개의 커널로 이루어진 Link Prediction map

![image](https://user-images.githubusercontent.com/70703320/115662422-ebf4ee80-a379-11eb-9901-b2f6590ed8c6.png)

- 1X1, 2(16) 형태의 레이어가 U-Net 구조로 연결되어 인접 pixel간 연결 구조가 지속적으로 유지되도록 하는 모델 구조

- 인접한 pixel이 중심 pixel과 단어단위로 연결된 pixel인지 아니면 분리된 pixel인지 알 수 있으므로, 문자영역이 단어단위로 분리된 Instance segmentation이 가능해


### 3) 최근의 방법들
딥러닝이 빠르게 발전하는 만큼 최신 연구들은 새로운 방식들을 계속해서 시도하고 있다.

#### 3-1) [CRAFT](https://arxiv.org/abs/1904.01941)
- Character 단위로 문자의 위치를 찾아낸 뒤 이를 연결하는 방식을 Segmentation기반으로 구현한 방법
- 문자의 영역을 boundary로 명확히 구분하지 않고 가우시안 분포를 따르는 원형의 score map을 만들어서 배치시키는 방법으로 문자의 영역을 학습
- 문자 단위 라벨을 가진 데이터셋이 많지 않기 때문에, 단어 단위의 정보만 있는 데이터셋에 대해 단어의 영역에 Inference를 한 후 얻어진 문자 단위의 위치를 다시 학습에 활용하는 Weakly supervised learning을 활용

#### 3-2) Pyramid Mask Text Detector
- Mask-RCNN의 구조를 활용하여 먼저 Text영역을 Region proposal network로 찾아냄
- Box head에서 더 정확하게 regression 및 classification을 하고 Mask head에서 Instance의 Segmentation을 하는 과정을 거침

![image](https://user-images.githubusercontent.com/70703320/115663395-61ad8a00-a37b-11eb-9ea4-6c9e152ea362.png)

- Mask 정보가 부정확한 경우를 반영하기 위해서 Soft-segmentation을 활용
- Mask-RCNN의 경우 단어 영역이 Box head에 의해 빨간색으로 잡히면 우측 처럼 boundary를 모두 Text 영역으로 잡지만, **PMTD는 단어의 사각형 배치 특성을 반영하여 피라미드 형태의 Score map을 활용**


## Text recognition


### 1) Unsegmented Data

- `unsegmented Data` ::  분리에 드는 비용이 비싸거나 어려워 Segmentation이 되어있지 않은 데이터를 Unsegmented data

#### 예시
- waveform 파일에서 어디까지가 각각 "the", "sound", "of"인지의 경계가 명확하게 분할되어 있지 않은 경우



### 2) [CNN과 RNN의 만남 CRNN](https://arxiv.org/pdf/1507.05717.pdf)

#### unsegmented data의 특징
- segment되어 있지 않은 하위데이터들끼리 시퀀스(sequence)를 이루고 있다는 점

#### CNN과 RNN의 만남 CRNN
- CNN(Convolutional neural network)과 RNN(Recurrent neural network)을 같이 쓰는 방법

#### 구체적인 방식
- 2-1) Feature Extractor로 사용되는 CNN을 기반의 VGG 또는 ResNet과 같은 네트워크로부터 문자의 정보를 가진 Feature를 얻어낼 수 있다.
- 2-2) 추출된 Feature를 Map-To-Sequence를 통해 Sequence형태의 feature로 변환한 후 다양한 길이의 Input을 처리할 수 있는 RNN으로 넣는다.
- 2-3) RNN이 Feature로 부터 문자를 인식하기 위해서는 문자 영역처럼 넓은 정보가 필요하기 때문에 LSTM으로 구성
- 2-4) 앞의 정보뿐만 아니라 뒤의 정보가 필요하기 때문에 이를 Bidirectional로 구성해서 Bidirectional LSTM을 사용


### 3) CTC
- 3-1) CRNN에서는 Step마다 Fully Connected Layer의 logit을 Softmax를 함으로써 어떤 문자일 확률이 높은지 알 수 있다. 

- 3-2) CRNN에서는 Unsegmented Data를 위해 CTC(Connectionist Temporal Classification)를 활용 (CTC는 Unsegmented data와 같이 Input과 Output이 서로 다른 Length의 Sequence가질 때, 이를 Align 없이 활용하는 방법)

- 3-3) CTC의 핵심인 모델의 Output에서 우리가 알고 싶어하는 Label sequence의 확률을 구할 수 있는 방법 

    - Label Encode에서 이렇게 같은 문자를 구분하기 위한 Blank를 중복된 라벨 사이를 구분하기 위해 넣어줌
    - Blank token을 '-'로 대신하여 Output을 만드는 Decoder를 의미합니다. Decode 후에 중복을 제거하고, 인식할 문자가 아닌 값을 지워주면 "HELLO"라는 결과를 얻을 수 있다.
    
![image](https://user-images.githubusercontent.com/70703320/115672394-7a6f6d00-a386-11eb-811e-e379c12d6dd2.png)

**Text의 정확도를 판별하기 위한 단위 중 실제 정답과 예측한 단어가 얼마나 가까운지 측정할 수 있는 방법**
- `Edit distance` ::  한국어로는 편집거리라고 하며 두 문자열 사이의 유사도를 판별하는 방법으로 예측된 단어에서 삽입, 삭제, 변경을 통해 얼마나 적은 횟수의 편집으로 정답에 도달할 수 있는지 최소 거리를 측정한다.



### 4) [TPS](https://arxiv.org/pdf/1603.03915.pdf)

![image](https://user-images.githubusercontent.com/70703320/115672682-c28e8f80-a386-11eb-80f5-b6979d5ce8e0.png)

- 책이 아닌 거리의 글자를 읽어낼 때, OCR이 어려워지는 이유는 아래 그림과 같이 불규칙한 방향이나 휘어진 진행방향 때문이다.

![image](https://user-images.githubusercontent.com/70703320/115672819-e520a880-a386-11eb-835e-748613f7e8af.png)

- Thin Plate Spline Transformation을 적용하면 입력이미지를 단어영역에 맞게 변형시켜 인식이 잘 되도록 해준다.

- Control point 20개를 미리 정의하고 Spatial Transformer Network를 통해서 Control point가 얼마나 움직여야 하는지 예측하는 네트워크를 아래 그림과 같이 Recognition model 앞단에 붙여 입력이미지를 정방향으로 맞춰준다.

![image](https://user-images.githubusercontent.com/70703320/115673029-2022dc00-a387-11eb-92a4-efe4cf20f120.png)

## Text recognition + Attention

- Attention과 Transformer가 OCR 분야에도 적용이 되었다.

### 1) Attention sequence prediction

![image](https://user-images.githubusercontent.com/70703320/115673533-b6570200-a387-11eb-994b-2e552aec4113.png)

- CTC를 활용한 CRNN의 경우, column에 따라서 prediction된 Label의 중복된 것들을 제거해줌으로써 우리가 원하는 형태의 label로 만들어주었다.

- Attention기반의 sequence prediction은 문장의 길이를 고정하고 입력되는 Feature에 대한 Attention을 기반으로 해당 글자의 Label을 prediction

- RNN으로 Character label을 뽑아낸다고 생각하면 되는데 첫번째 글자에서 입력 feature에 대한 Attention을 기반으로 label을 추정하고, 추정된 label을 다시 입력으로 사용하여 다음 글자를 추정해내는 방식


### 2) Transformer와 함께

![image](https://user-images.githubusercontent.com/70703320/115675805-e7d0cd00-a389-11eb-970b-db521e12f514.png)

- Irregular text를 잘 인식하기 위해서 2d space에 대한 attention을 활용하여 문자를 인식하기 위해 Transformer를 활용

- Query, Key, Value라는 개념을 통해서 Self-Attention을 입력으로부터 만들어낸다.

- 이를 통해 입력에서 중요한 Feature에 대해 Weight를 주게 된다.